In [146]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [147]:
data_dir = "./1D-AEMpy/"
depth_steps = 25 * 2 

print(os.getcwd())

C:\Users\au740615\Documents\Projects\1d_aempy\1D-AEMpy\mcl\1_trainingData


In [148]:
meterological_data_df = pd.read_csv("./../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1))   * num_time_steps)*0.5

depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
#meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
#meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,icemovAvg,density_snow,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior
1,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,6.955237,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,6.955237
2,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,7.366260,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,7.366260
3,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,7.745394,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,7.745394
4,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,8.096978,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,8.096978
5,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,8.427243,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,8.427243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,14.507095,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,14.507095
52556,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,14.531646,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,14.531646
52557,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,14.545849,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,14.545849
52558,2018-05-17 22:00:00,13.690588,-54.580442,-40.060336,-6.831580,0.000000,0.6,2.606367e+07,0.032069,39850000.0,...,14.550132,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,14.550132


In [149]:
## INPUT WTR

depth_list = np.array(list(range(1, depth_steps+1))   )*0.5

input_temp_df = pd.read_csv("./../output/py_temp_mix05.csv")

a = ['temp_input00_{}'.format(x) for x in depth_list]

flattened_inp_temp = input_temp_df.iloc[:-1,]

column_labels =['time']+ a

flattened_inp_temp.columns = column_labels 

input_temp_df =pd.DataFrame(flattened_inp_temp)

#flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
#data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_mix05':flattened_out_temp} #, 'depth':depth_list}
#input_temp_df = pd.DataFrame(data=data)


In [150]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_input00_20.5,temp_input00_21.0,temp_input00_21.5,temp_input00_22.0,temp_input00_22.5,temp_input00_23.0,temp_input00_23.5,temp_input00_24.0,temp_input00_24.5,temp_input00_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.474921,10.425209,10.400216,10.400101,10.400108,10.400158,10.400212,10.400541,10.400231,10.400343
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474854,10.425340,10.400352,10.400152,10.400163,10.400236,10.400319,10.400809,10.400349,10.400514
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474786,10.425473,10.400490,10.400203,10.400217,10.400315,10.400427,10.401076,10.400467,10.400684
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474717,10.425606,10.400628,10.400255,10.400272,10.400394,10.400535,10.401341,10.400586,10.400854
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474650,10.425738,10.400766,10.400307,10.400326,10.400473,10.400643,10.401605,10.400705,10.401024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117066,5.118292,5.119893,5.121992,5.122108,5.115242,5.079409,4.979856,4.782283,4.663583
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117108,5.118329,5.119934,5.122034,5.122134,5.115204,5.079258,4.979734,4.782716,4.664709
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117150,5.118366,5.119975,5.122076,5.122159,5.115165,5.079107,4.979613,4.783150,4.665830
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117193,5.118403,5.120016,5.122119,5.122185,5.115127,5.078957,4.979494,4.783585,4.666945


In [151]:
## 05 FINAL WTR

depth_list = np.array(list(range(1, depth_steps+1))   )*0.5

input_temp_df = pd.read_csv("./../output/py_temp_mix05.csv")

a = ['temp_mix05_{}'.format(x) for x in depth_list]

flattened_inp_temp = input_temp_df.iloc[1:,]

column_labels =['time']+ a

flattened_inp_temp.columns = column_labels 

input_temp_df =pd.DataFrame(flattened_inp_temp)

#
final_df = final_df.merge(input_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_mix05_20.5,temp_mix05_21.0,temp_mix05_21.5,temp_mix05_22.0,temp_mix05_22.5,temp_mix05_23.0,temp_mix05_23.5,temp_mix05_24.0,temp_mix05_24.5,temp_mix05_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.474921,10.425209,10.400216,10.400101,10.400108,10.400158,10.400212,10.400541,10.400231,10.400343
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474854,10.425340,10.400352,10.400152,10.400163,10.400236,10.400319,10.400809,10.400349,10.400514
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474786,10.425473,10.400490,10.400203,10.400217,10.400315,10.400427,10.401076,10.400467,10.400684
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474717,10.425606,10.400628,10.400255,10.400272,10.400394,10.400535,10.401341,10.400586,10.400854
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474650,10.425738,10.400766,10.400307,10.400326,10.400473,10.400643,10.401605,10.400705,10.401024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117066,5.118292,5.119893,5.121992,5.122108,5.115242,5.079409,4.979856,4.782283,4.663583
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117108,5.118329,5.119934,5.122034,5.122134,5.115204,5.079258,4.979734,4.782716,4.664709
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117150,5.118366,5.119975,5.122076,5.122159,5.115165,5.079107,4.979613,4.783150,4.665830
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117193,5.118403,5.120016,5.122119,5.122185,5.115127,5.078957,4.979494,4.783585,4.666945


In [152]:
buoyancy_data_df = pd.read_csv("./../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

#buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

#flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
#time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

#data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
#buoyancy_data_df = pd.DataFrame(data=data)

#buoyancy_data_df



a = ['buoyancy_{}'.format(x) for x in depth_list]

flattened_buoy = buoyancy_data_df.iloc[1:,]

column_labels =['time']+ a

flattened_buoy.columns = column_labels 

buoyancy_data_df =pd.DataFrame(flattened_buoy)

In [153]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,buoyancy_20.5,buoyancy_21.0,buoyancy_21.5,buoyancy_22.0,buoyancy_22.5,buoyancy_23.0,buoyancy_23.5,buoyancy_24.0,buoyancy_24.5,buoyancy_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,9.179640e-05,4.590698e-05,2.111539e-07,1.326507e-08,9.045154e-08,9.974311e-08,6.026277e-07,5.670420e-07,2.050182e-07,2.050182e-07
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,9.143034e-05,4.589956e-05,3.669534e-07,1.929729e-08,1.354379e-07,1.513686e-07,8.980005e-07,8.436618e-07,3.033287e-07,3.033287e-07
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,9.105858e-05,4.589134e-05,5.257162e-07,2.481667e-08,1.802876e-07,2.040023e-07,1.189912e-06,1.116261e-06,3.992120e-07,3.992120e-07
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,9.068759e-05,4.588234e-05,6.842082e-07,2.982840e-08,2.250093e-07,2.575810e-07,1.478548e-06,1.385055e-06,4.928108e-07,4.928108e-07
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,9.031900e-05,4.587259e-05,8.416342e-07,3.434273e-08,2.696078e-07,3.120692e-07,1.764003e-06,1.650154e-06,5.842024e-07,5.842024e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,4.297442e-07,5.617877e-07,7.378272e-07,4.082132e-08,2.408971e-06,1.233988e-05,3.223601e-05,5.503006e-05,2.731567e-05,2.731567e-05
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,4.280105e-07,5.632284e-07,7.383101e-07,3.500556e-08,2.431398e-06,1.237774e-05,3.222283e-05,5.488448e-05,2.718450e-05,2.718450e-05
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,4.262243e-07,5.646741e-07,7.387834e-07,2.916233e-08,2.453820e-06,1.241531e-05,3.220918e-05,5.473928e-05,2.705429e-05,2.705429e-05
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,4.243634e-07,5.661057e-07,7.392363e-07,2.327957e-08,2.476236e-06,1.245259e-05,3.219502e-05,5.459444e-05,2.692502e-05,2.692502e-05


In [154]:
out_diffusivity_df = pd.read_csv("./../output/py_diff.csv")

#out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

#flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
#time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

#data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
#out_diffusivity_df = pd.DataFrame(data=data)

#out_diffusivity_df



a = ['diffusivity_{}'.format(x) for x in depth_list]

flattened_diff = out_diffusivity_df.iloc[1:,]

column_labels =['time']+ a

flattened_diff.columns = column_labels 

out_diffusivity_df =pd.DataFrame(flattened_diff)

In [155]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,diffusivity_20.5,diffusivity_21.0,diffusivity_21.5,diffusivity_22.0,diffusivity_22.5,diffusivity_23.0,diffusivity_23.5,diffusivity_24.0,diffusivity_24.5,diffusivity_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07,2.450000e-07
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07,2.712500e-07
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07,2.778125e-07
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07,2.794531e-07
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07,2.798633e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,2.800389e-07,2.800270e-07,2.800187e-07,2.800130e-07,2.800090e-07,2.800063e-07,2.800044e-07,2.800031e-07,2.800021e-07,2.800015e-07
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,2.800137e-07,2.800094e-07,2.800064e-07,2.800044e-07,2.800030e-07,2.800021e-07,2.800014e-07,2.800010e-07,2.800007e-07,2.800005e-07
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,2.800042e-07,2.800028e-07,2.800019e-07,2.800013e-07,2.800009e-07,2.800006e-07,2.800004e-07,2.800003e-07,2.800002e-07,2.800001e-07
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,2.800012e-07,2.800008e-07,2.800005e-07,2.800003e-07,2.800002e-07,2.800002e-07,2.800001e-07,2.800001e-07,2.800000e-07,2.800000e-07


In [156]:
out_temp_df = pd.read_csv("./../output/py_temp_heat01.csv")



a = ['temp_heat01_{}'.format(x) for x in depth_list]

flattened_temp = out_temp_df.iloc[1:,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [157]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_heat01_20.5,temp_heat01_21.0,temp_heat01_21.5,temp_heat01_22.0,temp_heat01_22.5,temp_heat01_23.0,temp_heat01_23.5,temp_heat01_24.0,temp_heat01_24.5,temp_heat01_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.475008,10.425121,10.400128,10.400101,10.400108,10.400158,10.400211,10.400543,10.400230,10.400344
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474950,10.425244,10.400255,10.400151,10.400162,10.400236,10.400317,10.400812,10.400346,10.400515
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474883,10.425375,10.400391,10.400202,10.400217,10.400315,10.400424,10.401081,10.400463,10.400686
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474815,10.425508,10.400529,10.400254,10.400271,10.400394,10.400532,10.401347,10.400582,10.400857
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474747,10.425641,10.400667,10.400306,10.400326,10.400473,10.400640,10.401613,10.400700,10.401027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117054,5.118291,5.119891,5.122000,5.122136,5.115358,5.079666,4.980253,4.781964,4.662623
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117096,5.118328,5.119932,5.122042,5.122162,5.115320,5.079514,4.980128,4.782397,4.663755
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117138,5.118365,5.119973,5.122085,5.122188,5.115282,5.079363,4.980005,4.782831,4.664881
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117180,5.118402,5.120014,5.122127,5.122213,5.115244,5.079213,4.979884,4.783265,4.666002


In [158]:
out_temp_df = pd.read_csv("./../output/py_lakecharacteristics.csv")

print(out_temp_df.shape)

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step



(52560, 5)


In [159]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_heat01_22.5,temp_heat01_23.0,temp_heat01_23.5,temp_heat01_24.0,temp_heat01_24.5,temp_heat01_25.0,Volume_m2,Osgood,MaxDepth_m,MeanDepth_m
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.400108,10.400158,10.400211,10.400543,10.400230,10.400344,4.868375e+08,2.93527,25.75,13.21675
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.400162,10.400236,10.400317,10.400812,10.400346,10.400515,4.868375e+08,2.93527,25.75,13.21675
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.400217,10.400315,10.400424,10.401081,10.400463,10.400686,4.868375e+08,2.93527,25.75,13.21675
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.400271,10.400394,10.400532,10.401347,10.400582,10.400857,4.868375e+08,2.93527,25.75,13.21675
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.400326,10.400473,10.400640,10.401613,10.400700,10.401027,4.868375e+08,2.93527,25.75,13.21675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.122136,5.115358,5.079666,4.980253,4.781964,4.662623,4.868375e+08,2.93527,25.75,13.21675
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.122162,5.115320,5.079514,4.980128,4.782397,4.663755,4.868375e+08,2.93527,25.75,13.21675
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.122188,5.115282,5.079363,4.980005,4.782831,4.664881,4.868375e+08,2.93527,25.75,13.21675
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.122213,5.115244,5.079213,4.979884,4.783265,4.666002,4.868375e+08,2.93527,25.75,13.21675


In [160]:
out_temp_df = pd.read_csv("./../output/py_temp_ice02.csv")


a = ['temp_ice02_{}'.format(x) for x in depth_list]

flattened_temp = out_temp_df.iloc[1:,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [161]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_ice02_20.5,temp_ice02_21.0,temp_ice02_21.5,temp_ice02_22.0,temp_ice02_22.5,temp_ice02_23.0,temp_ice02_23.5,temp_ice02_24.0,temp_ice02_24.5,temp_ice02_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.475008,10.425121,10.400128,10.400101,10.400108,10.400158,10.400211,10.400543,10.400230,10.400344
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474950,10.425244,10.400255,10.400151,10.400162,10.400236,10.400317,10.400812,10.400346,10.400515
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474883,10.425375,10.400391,10.400202,10.400217,10.400315,10.400424,10.401081,10.400463,10.400686
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474815,10.425508,10.400529,10.400254,10.400271,10.400394,10.400532,10.401347,10.400582,10.400857
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474747,10.425641,10.400667,10.400306,10.400326,10.400473,10.400640,10.401613,10.400700,10.401027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117054,5.118291,5.119891,5.122000,5.122136,5.115358,5.079666,4.980253,4.781964,4.662623
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117096,5.118328,5.119932,5.122042,5.122162,5.115320,5.079514,4.980128,4.782397,4.663755
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117138,5.118365,5.119973,5.122085,5.122188,5.115282,5.079363,4.980005,4.782831,4.664881
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117180,5.118402,5.120014,5.122127,5.122213,5.115244,5.079213,4.979884,4.783265,4.666002


In [162]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [163]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [164]:
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_ice02_21.5,temp_ice02_22.0,temp_ice02_22.5,temp_ice02_23.0,temp_ice02_23.5,temp_ice02_24.0,temp_ice02_24.5,temp_ice02_25.0,day_of_year,time_of_day
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.400128,10.400101,10.400108,10.400158,10.400211,10.400543,10.400230,10.400344,140,1
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.400255,10.400151,10.400162,10.400236,10.400317,10.400812,10.400346,10.400515,140,2
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.400391,10.400202,10.400217,10.400315,10.400424,10.401081,10.400463,10.400686,140,3
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.400529,10.400254,10.400271,10.400394,10.400532,10.401347,10.400582,10.400857,140,4
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.400667,10.400306,10.400326,10.400473,10.400640,10.401613,10.400700,10.401027,140,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.119891,5.122000,5.122136,5.115358,5.079666,4.980253,4.781964,4.662623,137,18
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.119932,5.122042,5.122162,5.115320,5.079514,4.980128,4.782397,4.663755,137,19
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.119973,5.122085,5.122188,5.115282,5.079363,4.980005,4.782831,4.664881,137,20
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.120014,5.122127,5.122213,5.115244,5.079213,4.979884,4.783265,4.666002,137,21


In [165]:
temp_mix03_df = pd.read_csv("./../output/py_temp_diff03.csv")


a = ['temp_diff03_{}'.format(x) for x in depth_list]

flattened_temp = temp_mix03_df.iloc[1:,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [166]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_diff03_20.5,temp_diff03_21.0,temp_diff03_21.5,temp_diff03_22.0,temp_diff03_22.5,temp_diff03_23.0,temp_diff03_23.5,temp_diff03_24.0,temp_diff03_24.5,temp_diff03_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.474921,10.425209,10.400216,10.400101,10.400108,10.400158,10.400212,10.400541,10.400231,10.400343
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474854,10.425340,10.400352,10.400152,10.400163,10.400236,10.400319,10.400809,10.400349,10.400514
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474786,10.425473,10.400490,10.400203,10.400217,10.400315,10.400427,10.401076,10.400467,10.400684
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474717,10.425606,10.400628,10.400255,10.400272,10.400394,10.400535,10.401341,10.400586,10.400854
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474650,10.425738,10.400766,10.400307,10.400326,10.400473,10.400643,10.401605,10.400705,10.401024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117066,5.118292,5.119893,5.121992,5.122108,5.115242,5.079409,4.979856,4.782283,4.663583
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117108,5.118329,5.119934,5.122034,5.122134,5.115204,5.079258,4.979734,4.782716,4.664709
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117150,5.118366,5.119975,5.122076,5.122159,5.115165,5.079107,4.979613,4.783150,4.665830
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117193,5.118403,5.120016,5.122119,5.122185,5.115127,5.078957,4.979494,4.783585,4.666945


In [167]:
temp_conv03_df = pd.read_csv("./../output/py_temp_conv04.csv")


a = ['temp_conv04_{}'.format(x) for x in depth_list]

flattened_temp = temp_conv03_df.iloc[1:,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [168]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_conv04_20.5,temp_conv04_21.0,temp_conv04_21.5,temp_conv04_22.0,temp_conv04_22.5,temp_conv04_23.0,temp_conv04_23.5,temp_conv04_24.0,temp_conv04_24.5,temp_conv04_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.474921,10.425209,10.400216,10.400101,10.400108,10.400158,10.400212,10.400541,10.400231,10.400343
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474854,10.425340,10.400352,10.400152,10.400163,10.400236,10.400319,10.400809,10.400349,10.400514
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474786,10.425473,10.400490,10.400203,10.400217,10.400315,10.400427,10.401076,10.400467,10.400684
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474717,10.425606,10.400628,10.400255,10.400272,10.400394,10.400535,10.401341,10.400586,10.400854
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474650,10.425738,10.400766,10.400307,10.400326,10.400473,10.400643,10.401605,10.400705,10.401024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117066,5.118292,5.119893,5.121992,5.122108,5.115242,5.079409,4.979856,4.782283,4.663583
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117108,5.118329,5.119934,5.122034,5.122134,5.115204,5.079258,4.979734,4.782716,4.664709
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117150,5.118366,5.119975,5.122076,5.122159,5.115165,5.079107,4.979613,4.783150,4.665830
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117193,5.118403,5.120016,5.122119,5.122185,5.115127,5.078957,4.979494,4.783585,4.666945


In [169]:
temp_initial00_df = pd.read_csv("./../output/py_temp_initial00.csv")


a = ['temp_initial00_{}'.format(x) for x in depth_list]

flattened_temp = temp_initial00_df.iloc[1:,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [170]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,temp_initial00_20.5,temp_initial00_21.0,temp_initial00_21.5,temp_initial00_22.0,temp_initial00_22.5,temp_initial00_23.0,temp_initial00_23.5,temp_initial00_24.0,temp_initial00_24.5,temp_initial00_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.474979,10.425086,10.400089,10.400050,10.400054,10.400079,10.400106,10.400271,10.400115,10.400172
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.474921,10.425209,10.400216,10.400101,10.400108,10.400158,10.400212,10.400541,10.400231,10.400343
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.474854,10.425340,10.400352,10.400152,10.400163,10.400236,10.400319,10.400809,10.400349,10.400514
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.474786,10.425473,10.400490,10.400203,10.400217,10.400315,10.400427,10.401076,10.400467,10.400684
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.474717,10.425606,10.400628,10.400255,10.400272,10.400394,10.400535,10.401341,10.400586,10.400854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,5.117024,5.118255,5.119851,5.121949,5.122081,5.115279,5.079560,4.979981,4.781850,4.662450
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,5.117066,5.118292,5.119893,5.121992,5.122108,5.115242,5.079409,4.979856,4.782283,4.663583
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,5.117108,5.118329,5.119934,5.122034,5.122134,5.115204,5.079258,4.979734,4.782716,4.664709
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,5.117150,5.118366,5.119975,5.122076,5.122159,5.115165,5.079107,4.979613,4.783150,4.665830


In [171]:
temp_obs_df = pd.read_csv("./../output/py_observed_temp.csv")

input_temp_df = pd.read_csv("./../output/py_temp_mix05.csv")
flattened_inp_temp = input_temp_df.iloc[1:,1:]

a = ['obs_temp_{}'.format(x) for x in depth_list]

flattened_temp = temp_obs_df.iloc[1:,1:]


flattened_temp[flattened_temp == -999] = flattened_inp_temp

column_labels =['time']+ a


flattened_temp.insert(0, 'time', temp_obs_df.iloc[1:,0])
#flattened_temp = {'time': temp_obs_df.iloc[1:,0], flattened_temp}

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [172]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,obs_temp_20.5,obs_temp_21.0,obs_temp_21.5,obs_temp_22.0,obs_temp_22.5,obs_temp_23.0,obs_temp_23.5,obs_temp_24.0,obs_temp_24.5,obs_temp_25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.209,10.428667,10.581667,10.581667,10.581667,10.581667,10.581667,10.581667,10.513542,10.594643
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.207,10.430333,10.583333,10.583333,10.583333,10.583333,10.583333,10.583333,10.514583,10.596429
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.204,10.432000,10.585000,10.585000,10.585000,10.585000,10.585000,10.585000,10.515625,10.598214
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.214,10.433667,10.586667,10.586667,10.586667,10.586667,10.586667,10.586667,10.516667,10.600000
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.240,10.435333,10.588333,10.588333,10.588333,10.588333,10.588333,10.588333,10.517708,10.601786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52553,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,7.040,7.420111,7.165000,7.136667,7.086667,7.086667,7.065000,7.065000,7.065000,7.069643
52554,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,7.054,7.417222,7.165833,7.137778,7.087778,7.087778,7.065833,7.065833,7.065833,7.070536
52555,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,7.059,7.414333,7.166667,7.138889,7.088889,7.088889,7.066667,7.066667,7.066667,7.071429
52556,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,7.042,7.411444,7.167500,7.140000,7.090000,7.090000,7.067500,7.067500,7.067500,7.072321


In [173]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../output/py_observed_temp.csv")

a = ['input_obs_temp{}'.format(x) for x in depth_list]

flattened_temp = temp_obs_df.iloc[:-1,]

column_labels =['time']+ a

flattened_temp.columns = column_labels 

out_temp_df =pd.DataFrame(flattened_temp)

In [174]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,input_obs_temp20.5,input_obs_temp21.0,input_obs_temp21.5,input_obs_temp22.0,input_obs_temp22.5,input_obs_temp23.0,input_obs_temp23.5,input_obs_temp24.0,input_obs_temp24.5,input_obs_temp25.0
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.209,10.428667,10.581667,10.581667,10.581667,10.581667,10.581667,10.581667,10.513542,10.594643
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.207,10.430333,10.583333,10.583333,10.583333,10.583333,10.583333,10.583333,10.514583,10.596429
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.204,10.432000,10.585000,10.585000,10.585000,10.585000,10.585000,10.585000,10.515625,10.598214
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.214,10.433667,10.586667,10.586667,10.586667,10.586667,10.586667,10.586667,10.516667,10.600000
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.240,10.435333,10.588333,10.588333,10.588333,10.588333,10.588333,10.588333,10.517708,10.601786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52565,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,7.040,7.420111,7.165000,7.136667,7.086667,7.086667,7.065000,7.065000,7.065000,7.069643
52566,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,7.054,7.417222,7.165833,7.137778,7.087778,7.087778,7.065833,7.065833,7.065833,7.070536
52567,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,7.059,7.414333,7.166667,7.138889,7.088889,7.088889,7.066667,7.066667,7.066667,7.071429
52568,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,7.042,7.411444,7.167500,7.140000,7.090000,7.090000,7.067500,7.067500,7.067500,7.072321


In [175]:
ice_data_df = pd.read_csv("./../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

#num_time_steps = ice_data_df.shape[0]
#depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
#depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
#ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
#ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
#print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time'])
final_df

,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,TKE_Jm-1,ShearStress_Nm-2,Area_m2,...,input_obs_temp22.0,input_obs_temp22.5,input_obs_temp23.0,input_obs_temp23.5,input_obs_temp24.0,input_obs_temp24.5,input_obs_temp25.0,ice,snow,snowice
0,2012-05-19 01:00:00,12.800326,-59.975532,-72.115828,-16.270492,0.000000,0.6,1.903681e+07,0.026008,39850000.0,...,10.581667,10.581667,10.581667,10.581667,10.581667,10.513542,10.594643,0.0,0.0,0.0
1,2012-05-19 02:00:00,12.140192,-56.638577,-70.510837,-18.766448,0.000000,0.6,1.770408e+07,0.024780,39850000.0,...,10.583333,10.583333,10.583333,10.583333,10.583333,10.514583,10.596429,0.0,0.0,0.0
2,2012-05-19 03:00:00,11.480188,-53.244302,-70.044739,-21.813652,0.000000,0.6,1.649237e+07,0.023636,39850000.0,...,10.585000,10.585000,10.585000,10.585000,10.585000,10.515625,10.598214,0.0,0.0,0.0
3,2012-05-19 04:00:00,10.820185,-49.665116,-70.307884,-25.217532,0.000000,0.6,1.540771e+07,0.022588,39850000.0,...,10.586667,10.586667,10.586667,10.586667,10.586667,10.516667,10.600000,0.0,0.0,0.0
4,2012-05-19 05:00:00,11.689755,-35.756041,-63.667974,-18.685219,0.000000,0.6,1.631201e+07,0.023464,39850000.0,...,10.588333,10.588333,10.588333,10.588333,10.588333,10.517708,10.601786,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52565,2018-05-17 18:00:00,21.270344,-24.437932,-11.382243,49.411118,241.950029,0.6,6.958788e+07,0.061716,39850000.0,...,7.136667,7.086667,7.086667,7.065000,7.065000,7.065000,7.069643,0.0,0.0,0.0
52566,2018-05-17 19:00:00,20.080342,-27.935416,-12.779405,39.432348,95.860102,0.6,6.022339e+07,0.056048,39850000.0,...,7.137778,7.087778,7.087778,7.065833,7.065833,7.065833,7.070536,0.0,0.0,0.0
52567,2018-05-17 20:00:00,17.950598,-37.242353,-21.991974,22.847060,13.997190,0.6,4.665847e+07,0.047279,39850000.0,...,7.138889,7.088889,7.088889,7.066667,7.066667,7.066667,7.071429,0.0,0.0,0.0
52568,2018-05-17 21:00:00,15.820593,-46.138036,-31.039366,7.552953,0.003882,0.6,3.536746e+07,0.039306,39850000.0,...,7.140000,7.090000,7.090000,7.067500,7.067500,7.067500,7.072321,0.0,0.0,0.0


In [176]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of                       time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0      2012-05-19 01:00:00     12.800326     -59.975532   -72.115828   
1      2012-05-19 02:00:00     12.140192     -56.638577   -70.510837   
2      2012-05-19 03:00:00     11.480188     -53.244302   -70.044739   
3      2012-05-19 04:00:00     10.820185     -49.665116   -70.307884   
4      2012-05-19 05:00:00     11.689755     -35.756041   -63.667974   
...                    ...           ...            ...          ...   
52565  2018-05-17 18:00:00     21.270344     -24.437932   -11.382243   
52566  2018-05-17 19:00:00     20.080342     -27.935416   -12.779405   
52567  2018-05-17 20:00:00     17.950598     -37.242353   -21.991974   
52568  2018-05-17 21:00:00     15.820593     -46.138036   -31.039366   
52569  2018-05-17 22:00:00     13.690588     -54.580442   -40.060336   

       Sensible_Wm-2  Shortwave_Wm-2  lightExtinct_m-1      TKE_Jm-1  \
0         -16.270492        0.000

In [177]:
#final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [178]:
final_df_null.to_csv("all_data_lake_modeling_in_time_perRow.csv", index=False)

In [179]:
# iterating the columns
for col in final_df_null.columns:
    print(col)

time
AirTemp_degC
Longwave_Wm-2
Latent_Wm-2
Sensible_Wm-2
Shortwave_Wm-2
lightExtinct_m-1
TKE_Jm-1
ShearStress_Nm-2
Area_m2
CC
ea
Jlw
Uw
Pa
RH
PP
IceSnowAttCoeff
iceFlag
icemovAvg
density_snow
ice_prior
snow_prior
snowice_prior
rho_snow_prior
IceSnowAttCoeff_prior
iceFlag_prior
dt_iceon_avg_prior
icemovAvg_prior
temp_input00_0.5
temp_input00_1.0
temp_input00_1.5
temp_input00_2.0
temp_input00_2.5
temp_input00_3.0
temp_input00_3.5
temp_input00_4.0
temp_input00_4.5
temp_input00_5.0
temp_input00_5.5
temp_input00_6.0
temp_input00_6.5
temp_input00_7.0
temp_input00_7.5
temp_input00_8.0
temp_input00_8.5
temp_input00_9.0
temp_input00_9.5
temp_input00_10.0
temp_input00_10.5
temp_input00_11.0
temp_input00_11.5
temp_input00_12.0
temp_input00_12.5
temp_input00_13.0
temp_input00_13.5
temp_input00_14.0
temp_input00_14.5
temp_input00_15.0
temp_input00_15.5
temp_input00_16.0
temp_input00_16.5
temp_input00_17.0
temp_input00_17.5
temp_input00_18.0
temp_input00_18.5
temp_input00_19.0
temp_input00_19.5
tem

In [180]:
print(final_df_null.shape)

(52570, 588)
